Problem https://projecteuler.net/problem=956

# Main idea

The prime factorization of $N\bigstar$ can be easily calculated. First of all, we can say that:

$N\bigstar = \prod_{i=2}^{N}i^{\sum_{j=1}^{N + 1 - i}j}$

$N\bigstar = \prod_{i=2}^{N}i^{\frac{(N + 1 - i)(N + 2 - i)}{2}}$

With this in mind, the prime factorization will be calculated obtaining the prime factorization of the first N numbers, and constructing the factorization based on the exponent of the i and the prime facotrization of i.

This is done on the `prime_decomp_for_sdfact` function, which return the exponents of each prime that construct $N\bigstar$.


Having this, we can have all the divisors selecting some exponents for each prime (the exponent must be leq than the prime exponent on the factorization) $D$ of $N\bigstar$, and also, the $\Omega(D)$.
In this case we only care of the divisors which its $\Omega$ is divisible by $N$.

So lets imagine that we must find all the divisors $D$ that have only one prime factor $P$ (with any multiplicity) that satisfy the need of $\Omega(D)\mod N \equiv 0$. 

Lets say that $P$ is present $P_i$ times in the prime factorization of $N\bigstar$

It is easy to note that all the divisors formed only by $P$ as prime factor are of the form $P^{\alpha N}$ with $0 \leq \alpha \leq P_i / N$.

So the sum of all this divisors is $S_P = 1 + P^N  + P^{2N}... + P^{(P_i // N) \times N}$

We can expand this solution thinking that we can use 2 primes $P$ and $Q$, in this case, the multiplicity of $P$ and $Q$ must add any number that can be divided by $N$. So, all the previous solution still work, but we can also add all divisors that are of the form $PQ^{\alpha N}$ with $0 \leq \alpha \leq Q_i / N$, so now the sum of all the divisors is $S_{PQ} = (1 + Q^N  + Q^{2N}... + Q^{(Q_i // N) \times N}) \times S_P$.

As you may notice, this is not complete, because other divisors that only include $P$ and $Q$ may satisfy $\Omega(D)\mod N \equiv 0$, for example, $P^2Q^{N-2}$ and $P^{3N + 2}Q^{2N-2}$. These two cases are similar, because in both cases P adds $2 \mod N$ to the $\Omega$ function of the divisor.


With that in mind, we can solve the problem by induction, lets say that we order all the primes, first we only use one prime to for every

To solve this, we can use Dynamic Programing.



In [97]:
from typing import List, Dict
from collections import Counter
from functools import cache


In [98]:
def get_prime_factors(up_to: int) -> List[List[int]]:
    prime_factors = [[] for _ in range(up_to + 1)]

    for num in range(2, up_to + 1):
        if not prime_factors[num]:
            prime_factors[num] = [num]
            for i in range(2, up_to // num + 1):
                if not prime_factors[num * i]:
                    prime_factors[num * i] = [num, i]
    for num in range(2, up_to + 1):
        prime_factors[num] = [
            n
            for factor in prime_factors[num]
            for n in prime_factors[factor]
        ]

    return prime_factors

In [99]:
def fast_exp(base: int, exp: int, module: int) -> int:
    return pow(base, exp, module)


In [100]:
def get_sum_until(up_to: int) -> int:
    return int((up_to * (up_to + 1)) / 2)

In [101]:
def prime_decomp_for_sdfact(n: int) -> Dict[int, int]:
    decomp = Counter()
    prime_factors = get_prime_factors(n)
    compressed_factors = [Counter(factors) for factors in prime_factors]
    for fact in range(2, n + 1):
        # get the number of times that the fact is present in the sdfact
        times_appeared = get_sum_until(n + 1 - fact)
        for prime_factor, multi in compressed_factors[fact].items():
            decomp[prime_factor] += multi * times_appeared
    return decomp

In [102]:
@cache
def smart_geometric_sum(prime: int, module: int, max_exp: int, max_value: int) -> int:
    prime_pow_module = fast_exp(prime, module, max_value)
    inv = pow(prime_pow_module -1, -1, max_value)
    
    return inv * (pow(prime_pow_module, max_exp + 1, max_value) - 1)
    

# Explanation of get_sum_of_prime_factor_options

Note that the result is set in $\bmod max\_value$ (Different from module, module says the module of the exponent, and max_value is the mod from the result)

We want to get the sum of all the options that can be written as $prime^n$ $\equiv$ k $\bmod$ module.

This is equivalent to: $prime^k$ + $prime^{k + module}$ + $prime^{k + 2module}$... + $prime^{k + ((multiplicity - k) // module) \times module}$

Writing $t=((multiplicity - k) // module)$

which is equivalent to: $prime^k \times \sum_{i=0}^{t} (prime^{module})^{i}$

Writing $pm=prime^{module}$

Equivalent to: $prime^k \times \frac{pm^{t + 1} - 1}{pm - 1}$

This can be calculated with fast_exp in log(multiplicity) time

**Note:** I failed with this approach, because i don't know how to get rid of the (pm - 1), and the division on mod aritmetic is weird.
For this reason, I use a less optimum way, that make the $prime^k \times \sum_{i=0}^{t} (prime^{module})^{i}$, reusing the $\sum_{i=0}^{t} (prime^{module})^{i}$ part which is the same for every k (t may have 2 different values depending on k).

**Note of the note:** I can get rid of the pm - 1 calculating the inverse (inv) with the pow function, making the result as $prime^k \times (pm^{t + 1} - 1)\times inv$


In [95]:
def get_sum_of_prime_factor_options(prime: int, k: int, multiplicity: int, module: int, max_value: int) -> int:
    t = (multiplicity - k) // module
    return (fast_exp(prime, k, max_value) * smart_geometric_sum(prime, module, t, max_value)) % max_value
    

In [83]:
def deprectaed_get_sum_of_prime_factor_options(prime: int, k: int, multiplicity: int, module: int, max_value: int ) -> int:
    t = (multiplicity - k) // module
    prime_pow_module = fast_exp(prime, module, max_value)
    mult = ((fast_exp(prime, k, max_value)) * (fast_exp(prime_pow_module, t + 1, max_value) - 1))
    inv = pow(prime_pow_module - 1, -1, max_value)
    return (mult * inv) % max_value
    

In [7]:
def modular_subst(minuend: int, subtrahend: int, module: int) -> int:
    if minuend >= subtrahend:
        return minuend - subtrahend
    return module - subtrahend + minuend

In [85]:
def get_result(n: int, module: int) -> int:
    prime_decomp = tuple(prime_decomp_for_sdfact(n).items())
    # dp[i][j] represent the sum of all the options to sum j mod n using from the i-eth prime until the last
    dp = [[0 for _ in range(n)] for _ in range(len(prime_decomp))]
    max_previous_sum = 0
    
    for i in range(len(prime_decomp) - 1, -1, -1):
        multiplicity = prime_decomp[i][1]
        
        for j in range(min(n, multiplicity + max_previous_sum + 1)):
            
            if i == len(prime_decomp) - 1:
                dp[i][j] = get_sum_of_prime_factor_options_not_opt(prime_decomp[i][0], j, multiplicity, n, module)
            else:
                ij_sum = 0
                for k in range(min(n, multiplicity + 1)):
                    needed_next_i = modular_subst(j, k, n)
                    if needed_next_i > max_previous_sum:
                        continue
                    
                    ij_sum += (dp[i+1][needed_next_i] * get_sum_of_prime_factor_options_not_opt(prime_decomp[i][0], k, multiplicity, n, module)) % module
                dp[i][j] = ij_sum % module
                if i == 0 and j == 0:
                    return dp[0][0]
        max_previous_sum = j
    return dp[0][0]

In [8]:
prime_decomp_for_sdfact(1000)[2]

164920562

In [30]:
get_sum_of_prime_factor_options_not_opt(2, 0, 8, 4, 999999001)

273

In [93]:
ini = time()
print(get_result(1000, 999999001))
print(f'Took {time() - ini:2f} seconds')


882086212
Took 248.177640 seconds


In [96]:
ini = time()
print(get_result(1000, 999999001))
print(f'Took {time() - ini:2f} seconds')

882086212
Took 248.743882 seconds


In [40]:
6368195719791280 % 999999001

81625078

In [73]:
ini = time()
print(fast_exp(2, 164920562, 999999001))
print(f'Took {time() - ini:2f} seconds')

851778183
Took 0.000000 seconds
